## GRAY IMAGES

In [1]:
from PIL import Image
import pandas as pd
import os

In [2]:
anot_path = "./data/attention.csv"
images_path = "./data/datasets/"

gray_images_path = "./data/grayimage"

In [3]:
df = pd.read_csv(anot_path)
df.columns = ['filename', 'score']
df['score'].value_counts()

score
4    68
1    43
2    36
3    32
0    18
5     1
Name: count, dtype: int64

In [4]:
save_path = "data/gray_data.csv"

data = {
    'width': [],
    'height': [],
    'file_path': [],
    'score': []
}

if not os.path.exists(gray_images_path):
    os.makedirs(gray_images_path)

for item, score in zip(df['filename'].tolist(), df['score'].tolist()):
    file_path = os.path.join(images_path, item)
    gray_path = os.path.join(gray_images_path, item)

    img = Image.open(file_path).convert("L")
    
    img.save(gray_path)
    
    data['width'].append(img.size[0])
    data['height'].append(img.size[1])
    data['file_path'].append(gray_path)
    data['score'].append(score)

In [5]:
pd.DataFrame(data).to_csv(save_path, index=False)

## BLUR CANNY CONTOUR 

In [6]:
import pandas as pd
from PIL import Image
import random
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

# anomali data 69, 5, 36
# rd_idx = 5

file_path = "data/gray_data.csv"
df = pd.read_csv(file_path)

def Proccess(df = df, idx = None, biar_rapih = False):
    if idx == None:
        idx = random.randint(0, len(df['file_path'].tolist()))
    
    sample_image_name = df['file_path'].tolist()[idx]
    score = df['score'].tolist()[idx]

    img_np = np.array(Image.open(sample_image_name))
    
    if not biar_rapih:
        # gausah didebug ini [biar apa biarin | pusing gw njir ngeliatnya]
        print(f"image idx: {idx} | score: {score} | filename: {sample_image_name}")
        print(f"image size: {img_np.shape}")

    blank = np.zeros(shape=img_np.shape, dtype='uint8')

    gaussBlur = cv.GaussianBlur(img_np, (175, 175), 0.3)
    canny = cv.Canny(gaussBlur, 100, 100)

    _, tresh = cv.threshold(img_np, 125, 255, cv.THRESH_BINARY)
    _, blurtresh = cv.threshold(gaussBlur, 125, 255, cv.THRESH_BINARY)

    contours, _ = cv.findContours(canny, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    cv.drawContours(blank, contours, -1, 255, 1)
    
    images = [img_np, gaussBlur, canny, tresh, blank]
    titles = ['Gray', 'Gaussian Blur', 'Canny', 'Threshold', 'Contours Drawn']
    
    return images, titles

def ShowImage(images, titles):
    
    plt.figure(figsize=(15, 8))

    for i in range(len(images)):
        plt.subplot(2, 3, i + 1)  # 2 baris, 3 kolom
        if len(images[i].shape) == 2:  # grayscale
            plt.imshow(images[i], cmap='gray')
        else:  # RGB image
            plt.imshow(cv.cvtColor(images[i], cv.COLOR_BGR2RGB))
        plt.title(titles[i])
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [7]:
max_w = max(df['width'].tolist())
min_w = min(df['width'].tolist())

max_h = max(df['height'].tolist())
min_h = min(df['height'].tolist())

print(f"max-width: {max_w} | min-width: {min_w} | max-height: {max_h} | min-height: {min_h}")

max-width: 1080 | min-width: 102 | max-height: 526 | min-height: 50


In [8]:
for i in range(0):
    images, titles = Proccess()
    ShowImage(images, titles)

## LOAD DATA INTO NDARRAY

In [9]:
import pandas as pd
import numpy as np
import cv2 as cv

file_path = "data/gray_data.csv"
df = pd.read_csv(file_path)

df.describe()

# mean width = 624
# mean height = 208

# biar 2^ pake width = 512 (2**9)
# biar 2^ pake height = 256 (2**8)

,width,height,score
count,198.000000,198.000000,198.000000
mean,624.005051,208.671717,2.464646
std,225.453530,99.118338,1.398426
min,102.000000,50.000000,0.000000
25%,468.500000,131.000000,1.000000
50%,646.000000,200.000000,3.000000
75%,795.500000,259.500000,4.000000
max,1080.000000,526.000000,5.000000


In [10]:
def resize(np_img, w = 512, h = 256):
    return cv.resize(np_img, dsize=(w, h), interpolation=cv.INTER_LINEAR)

In [11]:
gray_np = [] # gray images
gaussBlur_np = []
canny_np = []
tresh_np = []
contour_np = []

# load to array
for i in range(len(df['file_path'].tolist())):
    '''
    images = [
        img_np, gaussBlur, canny, tresh, blank]
        [gray, gaussblur(gray), canny(gray), tresh(gray), contour(grey)
    ]
    '''
    
    images, _ = Proccess(idx=i, biar_rapih=True)
    # print(type(resize(images[0])))
    # print(resize(images[0]))
    # print(resize(images[0]).shape)
    # break
    gray_np.append(resize(images[0]))
    gaussBlur_np.append(resize(images[1]))
    canny_np.append(resize(images[2]))
    tresh_np.append(resize(images[3]))
    contour_np.append(resize(images[4]))

# transform into 
gray_np = np.array(gray_np)
gaussBlur_np = np.array(gaussBlur_np)
canny_np = np.array(canny_np)
tresh_np = np.array(tresh_np)
contour_np = np.array(contour_np)

In [12]:
print(gray_np.shape, gaussBlur_np.shape, canny_np.shape, tresh_np.shape, contour_np.shape)

# pake dah mau pake yang mana

(198, 256, 512) (198, 256, 512) (198, 256, 512) (198, 256, 512) (198, 256, 512)


## TRY MODEL [alex]

### RANDOM FOREST

In [13]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [14]:
X = tresh_np.reshape(tresh_np.shape[0], -1)
X = X / 255
y = np.array(df['score']).reshape(-1, 1)

X.shape, y.shape

((198, 131072), (198, 1))

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
type(X_train[0]), X_train[0]

(numpy.ndarray, array([0., 0., 0., ..., 1., 1., 1.]))

In [17]:
type(y_train[0]), y_train[0]

(numpy.ndarray, array([3], dtype=int64))

In [18]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 8, 16, 64],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [5, 15, 20],
    'bootstrap': [True, False]
}

random_forest = RandomForestClassifier(random_state=42)
random_forest

RandomForestClassifier(random_state=42)

In [19]:
random_search = RandomizedSearchCV(
    estimator=random_forest,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 8, 16, 64],
                                        'min_samples_leaf': [5, 15, 20],
                                        'min_samples_split': [5, 10, 15],
                                        'n_estimators': [50, 100, 200]},
                   random_state=42, verbose=2)

In [20]:
best_params = random_search.best_params_
best_estimators = random_search.best_estimator_
best_score = random_search.best_score_

print(f"best scores: {best_score} \nbest parameters: {best_params}")

best scores: 0.35403225806451616 
best parameters: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 20, 'max_depth': 16, 'bootstrap': True}


In [21]:
best_estimators

RandomForestClassifier(max_depth=16, min_samples_leaf=20, min_samples_split=5,
                       random_state=42)

### CNN

In [22]:
import torch
import torch.nn.functional as F
from torch import optim
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [23]:
class CNN(nn.Module):
    def __init__(self, input_channels = 1, num_classes = 6):
        super(CNN, self).__init__()
        
        # input 1 => output 8 => output size = torch.Size([8, 512, 256])
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=8, kernel_size=3, stride=1, padding=1)
        
        # max pool => output size = torch.Size([8, 511, 255]) 
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)
        
        # input 8 => output size = torch.Size([16, 511, 255])
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        
        # fully connected layer => output size = torch.Size([6])
        self.fc1 = nn.Linear(2084880, num_classes)
    
    def forward(self, x):
        conv_result1 = self.conv1(x)
        pool_result = self.pool(conv_result1)
        conv_result2 = self.conv2(pool_result)
        flatten_result = torch.flatten(conv_result2, start_dim=1)
        fully_connected = self.fc1(flatten_result)
        
        return fully_connected    

In [24]:
# one-hot encode
y_one_hot_encode = []
for i in df['score'].tolist():
    onehot_temp = [0]*6
    onehot_temp[i] = 1
    y_one_hot_encode.append(onehot_temp)

In [25]:
X = tresh_np.reshape(-1, 1, 512, 256).astype(np.float32)
X = X / 255.0
y = np.array(y_one_hot_encode).astype(np.float32)

# convert data to tensor
X = torch.tensor(X)
y = torch.tensor(y, dtype=torch.long)

X.shape, y.shape

(torch.Size([198, 1, 512, 256]), torch.Size([198, 6]))

In [26]:
# model = CNN()

# sample_input = X[0].unsqueeze(0)

# conv1res = model.conv1(sample_input)
# poolres = model.pool(conv1res)
# conv2res = model.conv2(poolres)
# flattenres = torch.flatten(conv2res, start_dim=1)
# fcres = model.fc1(flattenres)

# print(sample_input.shape, conv1res.shape, poolres.shape, conv2res.shape, flattenres.shape, fcres.shape, model(sample_input).shape)
# print(fcres, y[0])

In [27]:
torch.tensor(y[0]).shape

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4188\4256222860.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y[0]).shape


torch.Size([6])

In [28]:
dataset = TensorDataset(X, y)
trainloader = DataLoader(dataset, batch_size=32, shuffle=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [29]:
model = CNN().to(device=device)

In [30]:
epochs = 10
learning_rate = 0.025
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [31]:
for epoch in range(epochs):
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(tqdm(trainloader)):
        data = data.to(device)
        target = target.to(device)
        
        # predict | forward
        pred_res = model(data)
        loss = criterion(pred_res, target)
        
        running_loss += loss.item()
                
        # bacward | calculate gradient
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
    print(f"epoch: {epoch} / {epochs}")
    avg_loss = running_loss / len(trainloader)
    print(f"Loss: {avg_loss:.4f}")

  0%|          | 0/7 [00:00<?, ?it/s]


RuntimeError: Expected floating point type for target with class probabilities, got Long

In [ ]:
torch.save(model.state_dict(), 'model/model1.pth')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset = TensorDataset(X, y)
trainloader = DataLoader(dataset, batch_size=32, shuffle=True)

loaded_model = CNN()
loaded_model.load_state_dict(torch.load('model/model1.pth'))
loaded_model.to(device)

loaded_model.eval()

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2084880, out_features=6, bias=True)
)

In [ ]:
rd_idx = random.randint(0, 198)

sample_input = X[rd_idx].unsqueeze(0)
sample_output = y[rd_idx]
print(sample_input.shape)

pred_result = loaded_model(sample_input)
print(pred_result, sample_output)

torch.Size([1, 1, 512, 256])


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [ ]:
def check_acc(loader, model, device):
    num_correct = 0
    num_samples = 0

    model.eval()  # Set model ke mode evaluasi
    with torch.no_grad():  # Tidak perlu track gradient
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)  # forward
            _, preds = scores.max(1)  # ambil index prediksi tertinggi

            num_correct += (preds == y.argmax(dim=1)).sum().item()  # kalau target one-hot
            num_samples += y.size(0)

    acc = num_correct / num_samples
    print(f"Accuracy: {acc*100:.2f}%")
    
    model.train()  # Kembalikan ke mode training setelah evaluasi


In [ ]:
check_acc(trainloader, loaded_model, device)

Accuracy: 23.23%


## EFFICIENT NET B0

In [48]:
!nvidia-smi

Tue May  6 22:05:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.07                 Driver Version: 566.07         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   62C    P8             10W /   55W |     995MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [52]:
from os import listdir
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc
import cv2
import os
import torch
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms

In [50]:
image_data = './grayimage/*.jpg'

In [55]:
df = pd.read_csv('./data/gray_data.csv')
df

,width,height,file_path,score
0,147,72,./data/grayimage\2.png,2
1,102,72,./data/grayimage\3.png,1
2,291,130,./data/grayimage\4.png,4
3,770,342,./data/grayimage\5.png,1
4,1052,284,./data/grayimage\6.png,3
...,...,...,...,...
193,400,50,./data/grayimage\197.png,0
194,1080,252,./data/grayimage\198.png,4
195,684,84,./data/grayimage\199.png,0
196,530,334,./data/grayimage\200.png,4


In [47]:
# Load SavedModel as a layer
feature_extractor = TFSMLayer("./", call_endpoint="serving_default", trainable=False)

# Bangun model dengan Functional API
inputs = tf.keras.Input(shape=(600, 600, 3))  # Sesuaikan dengan input model kamu
x = feature_extractor(inputs)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

AttributeError: property 'trainable' of 'UninitializedVariable' object has no setter

In [ ]:
m.build([None, 512, 256, 1])